# Titanic Survival

### NOTES

No todas las tablas se usaran para el modelo de ML ya que algunas columnas solo contienen valores categoricos/informativos o columnas que contienen una gran cantidad de valores NULL.

Para el modelo se dropearan inicialmente las tablas de:
* PassengerId 
* Name
* Ticket
* Fare
* Cabin
* Embarked 

Y se usaran inicialmente las tablas de:
* Sex
* Age

In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import math

sns.set_theme(style="darkgrid")
df = pd.read_csv("train.csv")
display(df)

ModuleNotFoundError: No module named 'pandas'

## Quick Data Count

In [ ]:
totalPassengers = len(df)
totalM = df['Sex'].tolist().count('male')
totalF = df['Sex'].tolist().count('female')
print("Total of passengers: ", totalPassengers)
print("Total of male passengers: ", totalM)
print("Total of female passengers: ", totalF)

sns.countplot(x='Sex', data=df, palette='rocket')


## Survivors

In [ ]:
totalM1 = len(df[(df['Sex'] == 'male') & (df['Survived'] == 1)])
totalF1 = len(df[(df['Sex'] == 'female') & (df['Survived'] == 1)])
totalM0 = len(df[(df['Sex'] == 'male') & (df['Survived'] == 0)])
totalF0 = len(df[(df['Sex'] == 'female') & (df['Survived'] == 0)])
print("Total survivors: ", totalM1 + totalF1)
print("Total of male survivors: ", totalM1)
print("Total of female survivors: ", totalF1)
print("Total of male non survivors: ", totalM0)
print("Total of female non survivors: ", totalF0)

sns.countplot(x='Survived', data=df, hue='Sex', palette='rocket')

## Checking for blank values

In [ ]:
df.isna().sum()

In [ ]:
maleBlanks1 = len(df[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df.Age.isnull())])
femaleBlanks1 = len(df[(df['Sex'] == 'female') & (df['Survived'] == 1) & (df.Age.isnull())])
maleBlanks0 = len(df[(df['Sex'] == 'male') & (df['Survived'] == 0) & (df.Age.isnull())])
femaleBlanks0 = len(df[(df['Sex'] == 'female') & (df['Survived'] == 0) & (df.Age.isnull())])

print("Male survivors age blanks: ", maleBlanks1)
print("Female survivors age blanks: ", femaleBlanks1)
print("Male non survivors age blanks: ", maleBlanks0)
print("Female non survivors age blanks: ", femaleBlanks0)

In [ ]:
pd.DataFrame(data = [df.isna().sum() / df.shape[0] * 100], index = ["Blanks %"])

## Survivors per age

In [ ]:
totalMSKid = len(df[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df['Age'] <= 10)])
totalMSTeen = len(df[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df['Age'] > 10 ) & (df['Age'] <= 20 )])
totalMSAdult = len(df[(df['Sex'] == 'male') & (df['Survived'] == 1) & (df['Age'] > 20)])
print("Total of kid male survivors: ", totalMSKid)
print("Total of teen male survivors: ", totalMSTeen)
print("Total of adult male survivors: ", totalMSAdult)
print(totalMSKid+totalMSTeen+totalMSAdult)
print()

totalFSKid = len(df[(df['Sex'] == 'female') & (df['Survived'] == 1) & (df['Age'] <= 10)])
totalFSTeen = len(df[(df['Sex'] == 'female') & (df['Survived'] == 1) & (df['Age'] > 10 ) & (df['Age'] <= 20)])
totalFSAdult = len(df[(df['Sex'] == 'female') & (df['Survived'] == 1) & (df['Age'] > 20)])
print("Total of kid female survivors: ", totalFSKid)
print("Total of teen female survivors: ", totalFSTeen)
print("Total of adult female survivors: ", totalFSAdult)
print(totalFSKid+totalFSTeen+totalFSAdult)

#display(df.groupby("Age")["Survived"].mean())

In [ ]:
theta = [0, 1]
h = lambda x, theta: 1/(1 + math.exp(-(theta[0] + theta[1] * x)))
j_i = lambda x ,y, theta: y * math.log(h(x, theta)) + (1 - y) * math.log(1 - h(x, theta))



In [ ]:
# Male
male = df[(df['Sex'] == 'male')]
print(male)
x = male['Age']
y = male['Survived']

sns.regplot(x=x, y=y, data=male, logistic=True, ci=None)